
#Making Eigensoft Input Files
To convert the PLINK output from Stacks, which doesn't input directly to EIG6.0.1

1. Define paths to input and output files
2. PLINK.map file alteration 
3. PLINK.ped file alteration 
    Note: for this, you need to have a tab separated file with 2 columns (sample names, population ID) 

(4. Make an Eigensoft parameters file to use) -- in future


(Ultimate gaol = Stacks PLINK Output to EIGENSOFT PCA figures)

In [8]:
#Import packages 

#regular expressions 
import re 

#pandas for working with dataframes
import pandas as pd 

#numpy
import numpy as np

#csv reader
import csv

##1. Define paths to input and output files 

In [9]:
#NOTE: There is one spot where you'll have to put in a path and it's in the Bash cell where it uses a file that has populations for each sample

Plink_Map_File_Path = "/Users/eleanorbors/Desktop/Analysis/Lionfish_Stacks_Files/all_lionfish_stacks_1.35_42bp/n3M3/populations_no_miles_p14_r80/batch_100.plink.map"

Plink_Ped_File_Path = "/Users/eleanorbors/Desktop/Analysis/Lionfish_Stacks_Files/all_lionfish_stacks_1.35_42bp/n3M3/populations_no_miles_p14_r80/batch_100.plink.ped"

Finished_Map_Output_Path = "/Users/eleanorbors/Desktop/Analysis/Lionfish_Stacks_Files/all_lionfish_stacks_1.35_42bp/n3M3/populations_no_miles_p14_r80/EIG_batch_100_p14_r80_nomiles.map"

Finished_Ped_Output_Path = "/Users/eleanorbors/Desktop/Analysis/Lionfish_Stacks_Files/all_lionfish_stacks_1.35_42bp/n3M3/populations_no_miles_p14_r80/EIG_batch_100_p14_r80_nomiles.ped"

##2. PLINK.map file alteration 
1. Import to python, excluding first line. 
2. Change the chromosome number all to "1" to make EIGENSOFT happy. 

In [10]:
#Define f as the table from the PLINK.map file

f = pd.read_table(Plink_Map_File_Path, skiprows=1, header=None)

#Define your MapFile as a dataframe made from the PLINK.map table and name the columns 
MapFile = pd.DataFrame(data = f)
MapFile.columns  = ['chr', 'variant_ID', 'position','bp_coorinate']

#To check work, un-hash "MapFile" below:
#MapFile



In [11]:
#Use "pd.DataFrame.replace()" to replace "un" with "1"
#Name the new dataframe "MapFile_finished" do distinguish it from an un-manipulated dataframe

MapFile_finished = MapFile.replace(to_replace = 'un', value=1)

#To check work, un-hash "MapFile_finished" below:
#MapFile_finished

In [12]:
#Export altered file with proper chromosome code to location defined above as Finished_Map_Output_Path

MapFile_finished.to_csv(Finished_Map_Output_Path, sep ='\t', index=False, header=False)



## 3. PLINK.ped file alteration 

In [13]:
#Read in the PLINK.ped file as a table, with no header 

f = pd.read_table(Plink_Ped_File_Path, skiprows=1, header=None, dtype=object)

#Define a dataframe "PedFile"
PedFile = pd.DataFrame(data = f)

#Check that your file starts with the right indiviudal and that there are the right number of individuals (0 indexed!)
#PedFile

In [14]:
#make first column read "pop_#" instead of just a number 
#Note: make sure the data types are correct when using the ".replace" command. Here, PedFile[0] is an object. 

#PedFile[0] = PedFile[0].replace(to_replace = ['1', '2', ], value = ['pop_1', 'pop_2']) 

#Make the chromosome column read "1" straight down.  Length is length of dataframe 
PedFile[4] = [1] * len(PedFile.index)

#Un-has command below to check work:
#PedFile

##Tricky part = Need to make a list that is the population names as they match up with sample names. 


In [15]:
#Make a new file that will be the sample list output (for later steps)
csvfile = "./sample_list_output.csv"

#write your list of individuals to the csv file you made above to match order later
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in PedFile[1]:
        writer.writerow([val])

In [16]:
%%bash

#Use Bash to implement awk to search file for the population names associated with EKB #s. 

#Use Awk to get the list of populations to align with the correct order for the samples by searching in order of samlpe list
#Command taken from a stacks overflow answer

awk -F \t 'FNR==NR{a[$1]=$0;next}{print a[$1]}' /Users/eleanorbors/Desktop/Analysis/All_RAD_Pop_Data.txt sample_list_output.csv > sample_pop_map_correct_order.csv

#Unhash to see if you did it right 
#cat sample_pop_map_correct_order.csv

In [17]:
#Use CSV file to make a column in the dataframe PedFile

colnames = ['sample','pop']
populations = pd.read_csv("sample_pop_map_correct_order.csv", sep = '\t', names=colnames)


PedFile[5] = list(populations['pop'])

#print pd.unique(PedFile[5])

#Check your beautiful work 
#PedFile


##Finally, change all AGCT to 1234 and then save the new file

In [18]:
#Use "to_replace" and "value" to define lists of items to replace in the file 

PedFile_finished = PedFile.replace(to_replace = ['A', 'G', 'C', 'T'] , value=[1, 2, 3, 4])

#PedFile_finished

In [19]:
#Use predefined path to the output file to make your input to EIGENSOFT

PedFile_finished.to_csv(Finished_Ped_Output_Path, sep ='\t', index=False, header=False)

# 4. Make par file for Eigensoft  -- TO BE DONE AT A LATER TIME

